In [8]:
print(torch.version.cuda )

12.1


In [9]:
torch.cuda.is_available()

True

In [13]:
!nvcc --version

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/bin/bash: nvcc: command not found


In [11]:
!echo $PATH

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/usr/lib/jvm/java-8-openjdk-amd64/bin:/home/cdsw/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/conda/bin:/usr/lib/hadoop/bin:/usr/lib/hadoop-hdfs/bin


In [12]:
!echo $LD_LIBRARY_PATH

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib:/usr/local/cuda/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64


In [ ]:
#!pip install -q transformers torch accelerate autoawq

In [ ]:
#!pip install -q  --upgrade torchvision

  You can safely remove it manually.


In [7]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Thu Sep 12 06:51:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           On  | 00000000:00:1C.0 Off |                    0 |
| N/A   33C    P0              59W / 300W |   4353MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Karen_TheEditor_V2_STRICT_Mistral_7B-AWQ")

/home/cdsw/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load the model with AWQ quantization
model = AutoModelForCausalLM.from_pretrained("TheBloke/Karen_TheEditor_V2_STRICT_Mistral_7B-AWQ", torch_dtype=torch.float16)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the generation pipeline with the recommended settings
editor = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)




You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [5]:
# Example input text
text_to_correct = "This is a smaple sentence with some errors.We is misspelled few word"

# ChatML formatted input
input_text = f"<|system|>Please correct the spelling and grammatical errors in the following text.<|end|>\n" \
             f"<|user|>{text_to_correct}<|end|>\n" \
             f"<|assistant|>"


In [36]:
text_to_correct = 'Fix grammatical errors in this sentence: When I grow up, I start to understand what he said is quite     right. The correct value is 12.5  % and 3   rd position. The time is 10:30 am \
                   The length of the object is 845 mm and cost $1m. The sunrise happens at 0750 EST and visibily is 35 km. The right way to summarize and favor some advisors is still to be found'

# ChatML formatted input
# input_text = f"<|system|>You are a document reviewer for users in England and need to follow the following instructions to review and correct a text.\
#               1. Please correct the spelling and grammatical errors in the following text. \
#               2. Please use Australian English instead of American English for correcting spellings. For example color gets corrected to colour, favor to favour and rationalize to rationalise and \
#               rationalize should be spellt as rationalise \
#               3. Fix extra spaces if they have been created between words. \
#               4. Do not leave a space between a numeral and a symbol)(or lettter) in expressions using currency symbols or a percentage sign  \
#               for example 5  % should be 5% and 2 nd show be 2nd \
#               |end|>\n" \
#              f"<|user|>{text_to_correct}<|end|>\n" \
#              f"<|assistant|>"

input_text = f"<|im_start|>system: You are a document reviewer for users in England and need to follow the following instructions to review and correct a text.\
              1. Please correct the spelling and grammatical errors in the following text. \
              2. Please use Australian English instead of American English for correcting spellings. For example color gets corrected to colour, favor to favour and rationalize to rationalise and \
              rationalize should be spellt as rationalise \
              3. Fix extra spaces if they have been created between words. \
              4. Do not leave a space between a numeral and a symbol)(or lettter) in expressions using currency symbols or a percentage sign  \
              for example 5  % should be 5% and 2 nd show be 2nd \
              |im_end|>\n" \
             f"<|im_start|> User : Edit the following text for spelling and grammar mistakes : {text_to_correct}<|im_end|>\n" \
             f"<|im_start|assistant>"

In [6]:
# Generate text using the recommended settings
output = editor(input_text, max_length=512, temperature=0.7, top_p=0.1, top_k=40, repetition_penalty=1.18, num_return_sequences=1)

# Print the result
print("Original text:\n", text_to_correct)
print("Corrected text:\n", output[0]['generated_text'].replace("<|assistant|>", "").strip())


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/cdsw/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/cdsw/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/cds

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [31]:
# Let us try to use CoEdit

from transformers import AutoTokenizer, T5ForConditionalGeneration


tokenizer = AutoTokenizer.from_pretrained("grammarly/coedit-xl")
model = T5ForConditionalGeneration.from_pretrained("grammarly/coedit-xl")
text_to_correct = 'Fix grammatical errors in this sentence: When I grow up, I start to understand what he said is quite     right. The correct value is 12.5  % and 3   rd position. The time is 10:30 am \
                   The length of the object is 845 mm and cost $1m. The sunrise happens at 0750 EST and visibily is 35 km. The right way to summarize and favor some advisors is still to be found'

# ChatML formatted input
input_text = f"<|system|>You are a document reviewer for users in England and need to follow the following instructions to review and correct a text.\
              Please correct the spelling and grammatical errors in the following text. \
              Please use Australian English instead of American English for correcting spellings. For example color gets corrected to colour, favor to favour and rationalize to rationalise and \
              rationalize should be spellt as rationalise \
              Fix extra spaces if they have been created between words. \
              Do not leave a space between a numeral and a symbol)(or lettter) in expressions using currency symbols or a percentage sign  \
              for example 5  % should be 5% and 2 nd show be 2nd |end|>\n" \
             f"<|user|>{text_to_correct}<|end|>\n" \
             f"<|assistant|>"

input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_length=256)
edited_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [12]:
print(f' Input Text : {input_text}  \
        \n Output Text: {edited_text} '  )

 Input Text : <|system|>Please correct the spelling and grammatical errors in the following text.Also fix extra spaces if they have been created between words.               Do not leave a space between a numeral and a symbol)(or lettter) in expressions using currency symbols or a percentage sign                for example 5  % should be 5% and 2 nd show be 2nd |end|>
<|user|>Fix grammatical errors in this sentence: When I grow up, I start to understand what he said is quite     right. The correct value is 12.5  % and 3   rd position<|end|>
<|assistant|>          
 Output Text: When I grow up, I will start to understand what he said is quite right. The correct value is 12.5 % and 3 rd position.??|assistant|> 


# Spell checker with language tool

In [3]:
# !pip install language-tool-python

In [12]:
import language_tool_python
import time

tool = language_tool_python.LanguageTool('en-AU') # needed for the first time, first query takes more time then fraction of a second.

In [23]:
text = """The color of the neighbor's house is very pretty, but it doesn't fit well with the overall neighborhood. 
They're going to organize a neighborhood event to celebrate it's new look. 
The theater, which was recently modernized, is located nearby. Many people complimented the new sidewalk, but some criticised it because it's very far from the curb. 
Additionally, the flavor of the dessert was amazing, but I realized it was different from what we usually have. 
Thiss sentence has somee mistakes like repeatd words."""
start_time = time.time()

matches = tool.check(text)

execution_time = time.time() - start_time

print(f"Time taken : {execution_time}")

corrected_text = tool.correct(text)

for match in matches:
    print(f"\n Error: {match.context} | Issue: {match.message} | Suggestions: {match.replacements}")

Time taken : 0.11923837661743164

 Error: The color of the neighbor's house is very pretty,... | Issue: Possible spelling mistake. ‘color’ is American English. | Suggestions: ['colour']

 Error: The color of the neighbor's house is very pretty, but it doesn't ... | Issue: Possible spelling mistake. ‘neighbor’ is American English. | Suggestions: ['neighbour']

 Error: ...ut it doesn't fit well with the overall neighborhood.  They're going to organize a neighborh... | Issue: Possible spelling mistake. ‘neighborhood’ is American English. | Suggestions: ['neighbourhood']

 Error: ...overall neighborhood.  They're going to organize a neighborhood event to celebrate it's ... | Issue: Possible spelling mistake. ‘organize’ is American English. | Suggestions: ['organise']

 Error: ...ghborhood.  They're going to organize a neighborhood event to celebrate it's new look.  The ... | Issue: Possible spelling mistake. ‘neighborhood’ is American English. | Suggestions: ['neighbourhood']

 Error: ...a

In [19]:
print("\nCorrected Text:\n")
print(corrected_text)


Corrected Text:

The colour of the neighbour's house is very pretty, but it doesn't fit well with the overall neighbourhood. 
They're going to organise a neighbourhood event to celebrate its new look. 
The theatre, which was recently modernised, is located nearby. Many people complimented the new sidewalk, but some criticised it because it's very far from the curb. 
Additionally, the flavour of the dessert was amazing, but I realised it was different from what we usually have. 
This sentence has some mistakes like repeated words. Testing a new line ....
